---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np   

In [2]:
def combine(arr1,arr2):
    answer = np.zeros(arr1.shape,dtype=int)
    
    for i in range(arr1.shape[0]):
        left = arr1[i]
        right = arr2[i]
        
        if(np.sum(left)==0 and np.sum(right)==0):
            answer[i] = left
        elif(np.sum(left) == 0 and np.sum(right) != 0):
            answer[i] = right
        elif(np.sum(left) !=0 and np.sum(right) == 0):
            answer[i] = left
        elif(left[0] == 0 and left[2]!= 0 and np.sum(right) != 0):
            answer[i] = right
        # if they both have a sum not 0 meaning they both have info in the row, keep the left row
        else:
            answer[i] = left 
            
        
    return answer

In [3]:
def change(x):
    # Month, day, year
    
    nums = ['1','2','3','4','5','6','7','8','9','10','11','12','01','02','03','04','05','06','07','08','09']
    
    dic = {
        'Jan':1,
        'Feb':2,
        'Mar':3,
        'Apr':4,
        'May':5,
        'Jun':6,
        'Jul':7,
        'Aug':8,
        'Sep':9,
        'Oct':10,
        'Nov':11,
        'Dec':12
    }
    
    if x in dic:
        return dic[x]
    else:
        if x in nums:
            return x
        else:
            return 0

In [4]:
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc,columns=['text'])

# clean
months = ['(January)','(Janaury)','(February)','(March)','(April)','(May)','(June)','(July)','(August)','(September)','(October)','(November)','(December)','(Decemeber)']
for month in months:
    df['text'] = df['text'].str.replace(month, lambda x: x.groups()[0][:3])
    
    
print(df['text'][317])
print('**********')
print(df['text'][49])
print('**********')
print(df['text'][473])
print('**********')

# 459      0    1  1998
# 460      0    1  2012
# 461      0   16  1991
# 465      0   19  1976
# 467      0    1  2011
# 468      0    1  1997
# 479      0    1  1996
# 487      0    1  1992
# 488      0    1  1977

patterns = [r'(?P<Month>\d{2})/(?P<Day>\d{2})/(?P<Year>\d{4})',r'(?P<Month>\d{2})/(?P<Day>\d{2})/(?P<Year>\d{2})',
            r'(?P<Month>\d{1})/(?P<Day>\d{2})/(?P<Year>\d{2})',r'(?P<Month>\d{1})/(?P<Day>\d{1})/(?P<Year>\d{2})',
           r'(?P<Month>[A-Za-z]{3})-(?P<Day>\d{2})-(?P<Year>\d{4})',r'(?P<Month>\d{1})-(?P<Day>\d{2})-(?P<Year>\d{2})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2}), (?P<Year>\d{4})',
           r'(?P<Month>[A-Za-z]{3}). (?P<Day>\d{2}), (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2}) (?P<Year>\d{4})',
           r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}) (?P<Year>\d{4})',r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}). (?P<Year>\d{4})',
           r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}), (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})th, (?P<Year>\d{4})',
           r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})st, (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})nd, (?P<Year>\d{4})',
           r'(?P<Month>\d{2})/(?P<Year>\d{4})',r'(?P<Month>\d{1})/(?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Year>\d{4})',
           r'(?P<Year>\d{4})']

r'(?P<Month>[A-Za-z]{3}), (?P<Year>\d{4})'
final = df['text'].str.extract(patterns[0],expand=True)
final = final.fillna(0)
final = final.apply(pd.to_numeric)
final = final.values

for i in range(1,len(patterns)):
    addition = df['text'].str.extract(patterns[i],expand=True)

    addition = addition.fillna(0)
    if 'Month' in addition.columns:
        addition['Month'] = addition['Month'].apply(lambda x: change(x))

    addition = addition.apply(pd.to_numeric)

    if 'Day' not in addition.columns and 'Month' not in addition.columns:
        addition['Day'] = np.zeros(addition.shape[0],dtype=int)
        addition['Month'] = np.zeros(addition.shape[0],dtype=int)
        addition = addition.values
        for j in range(addition.shape[0]):
            row = addition[j]
            if np.sum(row) != 0:
                row[1] = 1
                row[2] = 1
                addition[j] = row
        addition = addition[:,[2,1,0]]

    elif 'Day' not in addition.columns:
        addition['Day'] = np.zeros(addition.shape[0],dtype=int)
        addition = addition.values
        for j in range(addition.shape[0]):
            row = addition[j]
            if np.sum(row) != 0:
                row[2] = 1
                addition[j] = row
        addition = addition[:,[0,2,1]]

    else:
        if (np.sum(addition.columns == ['Day','Month','Year']) == 3):
            addition = addition.values
            addition = addition[:,[1,0,2]]
        else:
            addition = addition.values

    final = combine(final,addition)

#change all years to 4 year format

for i in range(final.shape[0]):
    row = final[i]
    year = final[i][2]

    if year < 100:
        year_str = str(year)
        year_str = '19' + year_str
        year = int(year_str)
        row[2] = year
        final[i] = row


. Psychosocial: lives w/ father, looking to get own apartment. Lives in Saluda. Fa. has roommate. Parents divorced when pt 5 y/o old. Mother lives in saluda, see her often, drives hm to work. Relationship okay w/ dad and mom is okay too. Has one sibling, age 18 y/o. Get on well currently. Close family on mother's side. No family issues. Employment: Woriking 63 hours - "I love it." At construction worker. School: finished HS - dipolma, started working after HS. Likes job, makes good money. Peers: have friends, no issues, all friends are sober from opiates but drink/smoke "they distance from me when I use." Single currently. Stressors: biggest stress - Mar, 1975 - uncle passed from heroin OD. Pt was close do him, was mother's brother. Finances - getting better, owe lots of money to people/banks too.

**********
2/28/75 Other Adult Mental Health Outcomes Scales Used:

**********
shx of TBI (1975) ISO MVA.Medical History:

**********


In [5]:
def getDates():
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.DataFrame(doc,columns=['text'])

    # clean
    months = ['(January)','(Janaury)','(February)','(March)','(April)','(May)','(June)','(July)','(August)','(September)','(October)','(November)','(December)','(Decemeber)']
    for month in months:
        df['text'] = df['text'].str.replace(month, lambda x: x.groups()[0][:3])

    # 459      0    1  1998
    # 460      0    1  2012
    # 461      0   16  1991
    # 465      0   19  1976
    # 467      0    1  2011
    # 468      0    1  1997
    # 479      0    1  1996
    # 487      0    1  1992
    # 488      0    1  1977

    patterns = [r'(?P<Month>\d{2})/(?P<Day>\d{2})/(?P<Year>\d{4})',r'(?P<Month>\d{2})/(?P<Day>\d{1})/(?P<Year>\d{4})',r'(?P<Month>\d{1})/(?P<Day>\d{2})/(?P<Year>\d{4})',r'(?P<Month>\d{1})/(?P<Day>\d{1})/(?P<Year>\d{4})',
                r'(?P<Month>\d{2})/(?P<Day>\d{2})/(?P<Year>\d{2})',r'(?P<Month>\d{2})/(?P<Day>\d{1})/(?P<Year>\d{2})',
                r'(?P<Month>\d{1})/(?P<Day>\d{2})/(?P<Year>\d{2})',r'(?P<Month>\d{1})/(?P<Day>\d{1})/(?P<Year>\d{2})',
               r'(?P<Month>[A-Za-z]{3})-(?P<Day>\d{2})-(?P<Year>\d{4})',r'(?P<Month>\d{1})-(?P<Day>\d{2})-(?P<Year>\d{2})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2}), (?P<Year>\d{4})',
               r'(?P<Month>[A-Za-z]{3}). (?P<Day>\d{2}), (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2}) (?P<Year>\d{4})',
               r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}) (?P<Year>\d{4})',r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}). (?P<Year>\d{4})',
               r'(?P<Day>\d{2}) (?P<Month>[A-Za-z]{3}), (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})th, (?P<Year>\d{4})',
               r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})st, (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Day>\d{2})nd, (?P<Year>\d{4})',
               r'(?P<Month>\d{2})/(?P<Year>\d{4})',r'(?P<Month>\d{1})/(?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}) (?P<Year>\d{4})',r'(?P<Month>[A-Za-z]{3}), (?P<Year>\d{4})',
               r'(?P<Year>\d{4})']

    final = df['text'].str.extract(patterns[0],expand=True)
    final = final.fillna(0)
    final = final.apply(pd.to_numeric)
    final = final.values

    for i in range(1,len(patterns)):
        addition = df['text'].str.extract(patterns[i],expand=True)

        addition = addition.fillna(0)
        if 'Month' in addition.columns:
            addition['Month'] = addition['Month'].apply(lambda x: change(x))

        addition = addition.apply(pd.to_numeric)

        if 'Day' not in addition.columns and 'Month' not in addition.columns:
            addition['Day'] = np.zeros(addition.shape[0],dtype=int)
            addition['Month'] = np.zeros(addition.shape[0],dtype=int)
            addition = addition.values
            for j in range(addition.shape[0]):
                row = addition[j]
                if np.sum(row) != 0:
                    row[1] = 1
                    row[2] = 1
                    addition[j] = row
            addition = addition[:,[2,1,0]]

        elif 'Day' not in addition.columns:
            addition['Day'] = np.zeros(addition.shape[0],dtype=int)
            addition = addition.values
            for j in range(addition.shape[0]):
                row = addition[j]
                if np.sum(row) != 0:
                    row[2] = 1
                    addition[j] = row
            addition = addition[:,[0,2,1]]

        else:
            if (np.sum(addition.columns == ['Day','Month','Year']) == 3):
                addition = addition.values
                addition = addition[:,[1,0,2]]
            else:
                addition = addition.values

        final = combine(final,addition)

    #change all years to 4 year format

    for i in range(final.shape[0]):
        row = final[i]
        year = final[i][2]

        if year < 100:
            year_str = str(year)
            year_str = '19' + year_str
            year = int(year_str)
            row[2] = year
            final[i] = row

    return final

In [6]:
def merge(x):
    month = str(x['Month'])
    day = str(x['Day'])
    year = str(x['Year'])
    
    date = month + '/' + day + '/' + year
    return date

In [7]:
# once all days are all in same format you can just convert them to pandas datetime and use pandas datetime sorter
# to sort them in order based on original index

def date_sorter():
    dates = getDates()
    df = pd.DataFrame(dates,columns=['Month','Day','Year'])

    df['Date'] = df.apply(merge,axis=1)
    df['Date'] = pd.to_datetime(df['Date'])

    df = df.drop(['Month','Day','Year'],axis=1)
    
    df = df.sort_values(by='Date',axis=0)
    
    series = pd.Series(df.index)
    
    return series


In [8]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64